# Model Prototyping - Testing Estimators and Strategies

Este notebook demonstra como testar diferentes estimadores e estratégias de otimização.

**Objetivos:**
1. Testar estimadores robustos de μ e Σ
2. Comparar métodos de shrinkage (Ledoit-Wolf, Nonlinear, Tyler)
3. Prototipar estratégias de otimização
4. Visualizar fronteira eficiente
5. Avaliar sensibilidade a parâmetros

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Imports successful")

## 1. Carregar Dados (usamos cache do notebook anterior)

In [ ]:
# Load configuration and data (similar to notebook 01)
from itau_quant.config import load_config, UniverseConfig, PortfolioConfig
import yfinance as yf

# Load configs
universe_config = load_config(
    str(project_root / 'configs' / 'universe_arara.yaml'),
    UniverseConfig
)
portfolio_config = load_config(
    str(project_root / 'configs' / 'portfolio_arara_basic.yaml'),
    PortfolioConfig
)

# Download sample data
end_date = datetime.today()
start_date = end_date - timedelta(days=365 * 2)  # 2 years for faster testing

data = yf.download(
    tickers=universe_config.tickers[:10],  # Use subset for prototyping
    start=start_date,
    end=end_date,
    progress=False,
    auto_adjust=True
)

if isinstance(data.columns, pd.MultiIndex):
    prices = data['Close']
else:
    prices = data

prices = prices.dropna(how='all').ffill().bfill()
returns = prices.pct_change().dropna()

print(f"Data loaded: {len(returns)} days, {len(returns.columns)} assets")
print(f"Assets: {', '.join(returns.columns)}")

## 2. Testar Estimadores de Retornos Esperados (μ)

In [ ]:
from itau_quant.estimators.mu import mean_return, huber_mean

# Simple mean
mu_simple = mean_return(returns, method='simple')

# Huber robust mean
mu_huber, weights_huber = huber_mean(returns, c=1.5)

# Compare
comparison = pd.DataFrame({
    'Simple Mean': mu_simple * 252,
    'Huber Mean': mu_huber * 252,
    'Difference': (mu_huber - mu_simple) * 252
})

print("Expected Returns Comparison (annualized):")
print(comparison.to_string())

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(comparison))
width = 0.35

ax.bar(x - width/2, comparison['Simple Mean'], width, label='Simple Mean', alpha=0.8)
ax.bar(x + width/2, comparison['Huber Mean'], width, label='Huber Mean', alpha=0.8)

ax.set_xlabel('Asset', fontsize=11)
ax.set_ylabel('Expected Return (annualized)', fontsize=11)
ax.set_title('Expected Returns: Simple vs Huber', fontsize=13, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(comparison.index, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 3. Testar Estimadores de Covariância (Σ)

In [ ]:
from itau_quant.estimators.cov import (
    sample_cov,
    ledoit_wolf_shrinkage,
    nonlinear_shrinkage,
    tyler_m_estimator
)

# Test different covariance estimators
cov_sample = sample_cov(returns)
cov_lw, shrinkage_lw = ledoit_wolf_shrinkage(returns)
cov_nls = nonlinear_shrinkage(returns)
cov_tyler = tyler_m_estimator(returns, max_iter=50)

print(f"Ledoit-Wolf shrinkage parameter: {shrinkage_lw:.4f}")
print(f"\\nCondition numbers:")
print(f"  Sample cov:    {np.linalg.cond(cov_sample.values):.2e}")
print(f"  Ledoit-Wolf:   {np.linalg.cond(cov_lw.values):.2e}")
print(f"  Nonlinear:     {np.linalg.cond(cov_nls.values):.2e}")
print(f"  Tyler:         {np.linalg.cond(cov_tyler.values):.2e}")

# Compare diagonal elements (variances)
var_comparison = pd.DataFrame({
    'Sample': np.diag(cov_sample.values) * 252,
    'Ledoit-Wolf': np.diag(cov_lw.values) * 252,
    'Nonlinear': np.diag(cov_nls.values) * 252,
    'Tyler': np.diag(cov_tyler.values) * 252
}, index=cov_sample.index)

print("\\nVariances (annualized):")
print(var_comparison.to_string())

## 4. Testar Otimização Mean-Variance

In [ ]:
from itau_quant.optimization.core.mv_qp import solve_mean_variance, MeanVarianceConfig

# Setup optimization config
config = MeanVarianceConfig(
    risk_aversion=portfolio_config.risk_aversion,
    turnover_penalty=portfolio_config.turnover_penalty,
    lower_bounds=pd.Series(0.0, index=returns.columns),
    upper_bounds=pd.Series(portfolio_config.max_position, index=returns.columns),
    previous_weights=pd.Series(0.0, index=returns.columns),
    solver="CLARABEL"
)

# Solve with Ledoit-Wolf covariance
mu_annual = mu_simple * 252
cov_annual = cov_lw * 252

result = solve_mean_variance(mu_annual, cov_annual, config)

if result.summary.is_optimal():
    print("✅ Optimization successful!")
    print(f"Status: {result.summary.status}")
    print(f"Runtime: {result.summary.runtime:.3f}s")
    
    weights = result.weights
    active_weights = weights[weights > 0.001].sort_values(ascending=False)
    
    print(f"\\nPortfolio Weights:")
    for ticker, w in active_weights.items():
        print(f"  {ticker}: {w:.2%}")
    
    # Portfolio metrics
    port_return = float(mu_annual @ weights)
    port_vol = float(np.sqrt(weights @ cov_annual @ weights))
    sharpe = port_return / port_vol
    
    print(f"\\nPortfolio Metrics (ex-ante):")
    print(f"  Expected Return: {port_return:+.2%}")
    print(f"  Volatility:      {port_vol:.2%}")
    print(f"  Sharpe Ratio:    {sharpe:.2f}")
else:
    print(f"❌ Optimization failed: {result.summary.status}")

## 5. Comparar Estratégias Baseline

In [ ]:
from itau_quant.optimization.core.risk_parity import solve_risk_parity

# 1/N (Equal-weight)
w_equal = pd.Series(1/len(returns.columns), index=returns.columns)

# Risk Parity
w_rp = solve_risk_parity(cov_lw)

# Mean-Variance (já calculado)
w_mv = weights

# Compare portfolios
strategies = {
    '1/N': w_equal,
    'Risk Parity': w_rp,
    'Mean-Variance': w_mv
}

metrics = []
for name, w in strategies.items():
    ret = float(mu_annual @ w)
    vol = float(np.sqrt(w @ cov_annual @ w))
    sr = ret / vol if vol > 0 else 0
    metrics.append({
        'Strategy': name,
        'Return': ret,
        'Volatility': vol,
        'Sharpe': sr
    })

metrics_df = pd.DataFrame(metrics).set_index('Strategy')
print("Strategy Comparison (ex-ante):")
print(metrics_df.to_string())

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(metrics_df['Volatility'], metrics_df['Return'], s=200, alpha=0.7)

for idx, row in metrics_df.iterrows():
    ax.annotate(idx, (row['Volatility'], row['Return']), 
                xytext=(5, 5), textcoords='offset points', fontsize=11)

ax.set_xlabel('Volatility (annualized)', fontsize=12)
ax.set_ylabel('Expected Return (annualized)', fontsize=12)
ax.set_title('Risk-Return Profile: Strategy Comparison', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Conclusões

Este notebook demonstrou:

1. **Estimadores de μ**: Comparação entre média simples e Huber robusto
2. **Estimadores de Σ**: Teste de diferentes métodos de shrinkage
3. **Otimização MV**: Implementação funcional com constraints
4. **Estratégias baseline**: Comparação de 1/N, Risk Parity e Mean-Variance

**Próximos passos:**
- Ver notebook `03-results-analysis.ipynb` para análise de backtests completos
- Experimentar diferentes valores de risk_aversion e shrinkage